# ABECTO Tutorial

ABECTO is an **AB**ox **E**valuation and **C**omparison **T**ool for **O**ntologies. It allows to easily compare and evaluate two or more RDF knowledge bases regarding the contained information. This tutorial provides an introduction to the use of ABECTO.


## Preparation

Before we can start, we need to do a few preparation steps.

ABECTO is running as a HTTP REST service in the background. We will use some provided functions, which hide the raw HTTP requests.

In [ ]:
from abecto import *

First, we create some sample files that we will use in this tutorial.

In [ ]:
import tempfile

source1file1 = tempfile.TemporaryFile(mode = "w+")
source1file1.write("""
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :alice rdf:type   :Person ;
           rdfs:label "Alice" ;
           :pnr       "12345"^^xsd:integer .
""")
source1file1.seek(0)

source1file2 = tempfile.TemporaryFile(mode = "w+")
source1file2.write("""
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :bob rdf:type   :Person ;
         rdfs:label "Bob" ;
         :pnr       "67890"^^xsd:integer .
""")
source1file2.seek(0)

source2file1 = tempfile.TemporaryFile(mode = "w+")
source2file1.write("""
    BASE         <http://example.org/b/>
    PREFIX :     <http://example.org/b/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    :alice rdf:type   :Person ;
           rdfs:label "Alice" .

    :bob rdf:type   :Person ;
         rdfs:label "Bob" .

    :charlie rdf:type   :Person ;
             rdfs:label "Charlie" .
""")
source2file1.seek(0);

source3file1 = tempfile.TemporaryFile(mode = "w+")
source3file1.write("""
    BASE         <http://example.net/b/>
    PREFIX :     <http://example.net/b/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    :P001 rdf:type   :Person ;
           rdfs:label "Alice" .

    :P002 rdf:type   :Person ;
             rdfs:label "Charlie" .
""")
source3file1.seek(0);

Now, we start the service. **This might take a few seconds.**

In [ ]:
abecto = Abecto("http://localhost:8080/", "target/abecto.jar")
abecto.start()

After the service was started, we are ready to create our ontology evaluation and comparison project.

## Project Setup

First, we create a new ABECTO project. We can also give the project an arbitrary name.

In [ ]:
project = abecto.project("My Project")

Now, we see our new project in the list of projects.

In [ ]:
abecto.projects()

We can also request information of a certain project.

In [ ]:
project.info()

Furthermore, we can delete projects.

In [ ]:
trashProject = abecto.project("Trash Project")
trashProject.delete()
abecto.projects()

Now, we create a knowledge base object for each knwoledge base we want to include into the project.

In [ ]:
kb1 = project.knowledgeBase("Knowledge Base ABCDEFG")
kb2 = project.knowledgeBase("Knowledge Base HIJKLMN")
kb3 = project.knowledgeBase("Knowledge Base OPQRSTU")

The knowledge bases appear now in the list of knowledge bases of the project.

In [ ]:
project.knowledgeBases()

We can also request information of a certain knowledge base.

In [ ]:
kb1.info()

And we can delete knowledge bases.

In [ ]:
trashKB = project.knowledgeBase("Trash Knowledge Base")
trashKB.delete()
project.knowledgeBases()

In [ ]:
kb1source1 = kb1.source("RdfFileSourceProcessor")
kb1source2 = kb1.source("RdfFileSourceProcessor")
kb2source1 = kb2.source("RdfFileSourceProcessor")
kb3source1 = kb3.source("RdfFileSourceProcessor")

In [ ]:
kb1source1.info()

In [ ]:
project.steps()

In [ ]:
kb1source1.load(source1file1)
kb1source2.load(source1file2)
kb2source1.load(source2file1)
kb3source1.load(source3file1);

In [ ]:
personPattern = {"person": "?person <http://www.w3.org/2000/01/rdf-schema#label> ?label ."}
categories1 = kb1source1.plus(kb1source2).into("ManualCategoryProcessor", {"patterns": personPattern})
categories2 = kb2source1.into("ManualCategoryProcessor", {"patterns": personPattern})
categories3 = kb3source1.into("ManualCategoryProcessor", {"patterns": personPattern})

In [ ]:
mapping = categories1.plus(categories2).plus(categories3).into("JaroWinklerMappingProcessor", {"threshold": 0.9, "case_sensitive": False, "category": "person", "variables": ["label"]})

In [ ]:
mappingReport = mapping.into("MappingReportProcessor")

In [ ]:
project.runAndAwait()

In [ ]:
mappingReport.last().graph()

In [ ]:
mappingReport.last().graphAsDataFrame()

In [ ]:
mappingReport.info()

In [ ]:
mappingReport.last().report()